In [ ]:
!pip install --q convokit


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.2/183.2 kB 4.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 671.3/671.3 kB 37.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.4/300.4 kB 25.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.4/175.4 kB 15.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 5.1 MB/s eta 0:00:00


In [14]:
import convokit


In [ ]:
corpus = convokit.Corpus(filename=convokit.download("movie-corpus"))

No configuration file found at /root/.convokit/config.yml; writing with contents: 
# Default Backend Parameters
db_host: localhost:27017
data_directory: ~/.convokit/saved-corpora
default_backend: mem


In [13]:
pip install  --q scikit-surprise

In [16]:
import pandas as pd
from surprise import Dataset,Reader,SVD
from surprise.model_selection import train_test_split
from surprise import accuracy

In [17]:
print("Number of conversations:", len(corpus.conversations))
print("Number of users:", len(corpus.speakers))
print("Number of utterances:", len(corpus.utterances))

Number of conversations: 83097
Number of users: 9035
Number of utterances: 304713


In [20]:
import pandas as pd

# Create empty lists to store data
conversation_ids = []
movie_indices = []
movie_names = []
release_years = []
ratings = []
votes = []
genres = []
num_utterances = []

# Loop through conversations and extract data
for convo_id in corpus.get_conversation_ids():
    convo = corpus.get_conversation(convo_id)

    # Extract metadata from ConvoKitMeta object
    metadata = convo.meta

    # Append data to respective lists
    conversation_ids.append(convo_id)
    movie_indices.append(metadata['movie_idx'])
    movie_names.append(metadata['movie_name'])
    release_years.append(metadata['release_year'])
    ratings.append(metadata['rating'])
    votes.append(metadata['votes'])
    genres.append(metadata['genre'])
    num_utterances.append(len(convo.get_utterance_ids()))

# Create a DataFrame from the lists
data = {
    'Conversation ID': conversation_ids,
    'Movie Index': movie_indices,
    'Movie Name': movie_names,
    'Release Year': release_years,
    'Rating': ratings,
    'Votes': votes,
    'Genres': genres,
    'Number of Utterances': num_utterances
}

df = pd.DataFrame(data)



In [21]:
reader = Reader(rating_scale=(1, 10))


In [22]:
data=df[['Conversation ID','Movie Name','Rating']]

In [23]:
data.head()

,Conversation ID,Movie Name,Rating
0,L1044,10 things i hate about you,6.90
1,L984,10 things i hate about you,6.90
2,L924,10 things i hate about you,6.90
3,L870,10 things i hate about you,6.90
4,L866,10 things i hate about you,6.90


In [27]:
data = Dataset.load_from_df(data, reader)


In [28]:
trainset, testset = train_test_split(data, test_size=0.2,random_state=42)


In [29]:
model=SVD()

In [30]:
model.fit(trainset)

In [31]:
predictions=model.test(testset)

In [32]:
rmse=accuracy.rmse(predictions)
print(f'RMSE:{rmse:.2f}')

RMSE: 0.0640
RMSE:0.06


In [33]:
top_n = 3
user_recommendations = []

In [34]:
item_column_name = 'Movie Name'
all_item_ids = df[item_column_name].unique()

In [41]:
user_id='L870'
for item_id in all_item_ids:
    predicted_rating = model.predict(user_id, item_id).est
    user_recommendations.append((item_id, predicted_rating))

In [42]:
user_recommendations.sort(key=lambda x: x[1], reverse=True)


In [43]:
top_n = 3
print(f'Top {top_n} recommendations for user {user_id} :')
for item_id, predicted_rating in user_recommendations[:top_n]:
  print(f'Item {item_id}: Predicted Rating ={predicted_rating:.2f}')



Top 3 recommendations for user L870 :
Item neuromancer: Predicted Rating =9.22
Item the godfather: Predicted Rating =9.22
Item neuromancer: Predicted Rating =9.14
